In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
data = pd.read_csv('Aggregated User Data.csv')
data.head()

,UUID,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,...,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,00EABED2-271D-49D8-B599-1D4A09240601,1444079161,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
1,00EABED2-271D-49D8-B599-1D4A09240601,1444079221,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
2,00EABED2-271D-49D8-B599-1D4A09240601,1444079281,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
3,00EABED2-271D-49D8-B599-1D4A09240601,1444079341,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
4,00EABED2-271D-49D8-B599-1D4A09240601,1444079431,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2


In [3]:
data = data[(data['UUID'] == '0BFC35E2-4817-4865-BFA7-764742302A2D') | (data['UUID'] == '0A986513-7828-4D53-AA1F-E02D6DF9561B') | (data['UUID'] == '00EABED2-271D-49D8-B599-1D4A09240601')] 
data.drop(columns = ['timestamp'], inplace = True)

In [4]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

In [5]:
activities = ['label:FIX_walking', 'label:SITTING', 'label:SLEEPING', 'label:PHONE_ON_TABLE']

In [6]:
activity_data = interpolation(data[(data['label:FIX_walking'] == 1) | (data['label:SITTING'] == 1) | (data['label:SLEEPING'] == 1)]).reset_index()
label_data = interpolation(activity_data[activities]).reset_index()

In [7]:
label_data.drop(columns = ['index'], inplace = True)

activity_data = activity_data.loc[:,'raw_acc:magnitude_stats:mean':'raw_acc:3d:ro_yz']

In [8]:
label_data.head()

,label:FIX_walking,label:SITTING,label:SLEEPING,label:PHONE_ON_TABLE
0,0.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0
2,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0
4,0.0,1.0,0.0,1.0


In [9]:
unique_labelpairs = label_data.groupby(activities).size().reset_index(name='Count')
unique_labelpairs.head()

,label:FIX_walking,label:SITTING,label:SLEEPING,label:PHONE_ON_TABLE,Count
0,0.0,0.0,1.0,0.0,1169
1,0.0,0.0,1.0,1.0,453
2,0.0,1.0,0.0,0.0,2896
3,0.0,1.0,0.0,1.0,1764
4,1.0,0.0,0.0,0.0,728


In [10]:
activity_data.head()

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz
0,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,6.684605,5.043970,...,0.148988,0.002331,0.004614,-0.996790,0.003269,0.003521,0.003539,0.106920,0.516842,0.255494
1,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,6.684603,5.043367,...,0.207046,0.003557,0.005495,-0.996832,0.002489,0.003772,0.004177,-0.079483,0.357748,0.036252
2,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,6.684605,5.043599,...,0.186961,0.004180,0.005776,-0.996785,0.003567,0.004051,0.003681,0.122432,0.464881,0.222375
3,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,6.684605,5.043263,...,0.479430,0.004179,0.005551,-0.996836,0.004202,0.002919,0.003559,0.308841,0.773514,0.296194
4,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,6.683904,5.042779,...,0.202305,-0.008383,-0.000009,-0.994184,0.074612,0.023784,0.039512,-0.187758,-0.204381,0.062696


In [11]:
indices = label_data.groupby(activities).groups

In [12]:
for k in range(len(unique_labelpairs)):
    key = list(indices)[k]
    val = list(indices.values())[k]
    #print(key, val)
    
    if k == 0:
        data_with_indices = activity_data.iloc[list(indices.values())[k]].copy(deep = True)
        data_with_indices['Unique Label Pair Index'] = k
    else:
        current_data = activity_data.iloc[list(indices.values())[k]].copy(deep = True)
        current_data['Unique Label Pair Index'] = k
        
        data_with_indices = pd.concat([data_with_indices, current_data])
        
data_with_indices

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,Unique Label Pair Index
478,0.997478,0.001418,-0.001431,0.002539,0.996710,0.997557,0.998350,1.800587,6.684611,5.043453,...,0.001369,0.015572,-0.997354,0.001038,0.001313,0.001423,0.130205,0.143692,0.183355,0
479,0.997303,0.001148,0.000560,0.001507,0.996528,0.997288,0.998111,2.556438,6.684611,5.043375,...,0.001457,0.015367,-0.997182,0.000987,0.001127,0.001146,0.016236,-0.015922,-0.099538,0
480,0.996976,0.001247,-0.000412,0.001608,0.996182,0.996953,0.997861,2.719104,6.684611,5.043517,...,0.001267,0.016086,-0.996844,0.001021,0.001193,0.001245,-0.051118,-0.017367,-0.133502,0
481,0.996895,0.001172,-0.000609,0.001543,0.996101,0.996910,0.997705,2.538713,6.684611,5.043646,...,0.001313,0.015924,-0.996766,0.000991,0.001128,0.001171,-0.060276,-0.028098,-0.099209,0
482,0.997187,0.001197,0.000329,0.001570,0.996432,0.997121,0.997998,2.527216,6.684611,5.043647,...,0.001311,0.015887,-0.997058,0.000979,0.001136,0.001195,0.004222,-0.036610,-0.112272,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6820,0.989871,0.067119,0.099246,0.150556,0.977105,0.981139,0.985729,0.974973,6.682444,5.036262,...,0.552374,-0.805412,0.038265,0.119152,0.090855,0.081419,0.689755,0.222105,0.380224,4
6861,1.003197,0.087047,0.085149,0.153087,0.972808,0.999512,1.025152,1.806637,6.680908,5.041197,...,0.502751,0.106469,0.844585,0.112669,0.112608,0.105780,0.052065,-0.319796,-0.185053,4
6926,1.022369,0.057372,0.047823,0.106795,1.001885,1.019989,1.042060,1.760117,6.683045,5.042440,...,-0.148705,0.469393,0.889832,0.049875,0.063585,0.088057,-0.533838,0.538437,-0.583549,4
6988,1.028238,0.022944,0.029560,0.057903,1.024304,1.027962,1.031794,1.055626,6.684366,5.041948,...,-0.520750,0.057947,0.881174,0.045585,0.059979,0.033450,0.505155,0.596598,0.229713,4


In [13]:
X = data_with_indices.iloc[:,:-1].values
y = data_with_indices.iloc[:,-1].values

In [14]:
idx_of_largest = unique_labelpairs['Count'].idxmax()
largest_volume = unique_labelpairs['Count'].max()
print(idx_of_largest, largest_volume)

2 2896


In [15]:
volume_dictionary = {}

In [16]:
for k in range(len(unique_labelpairs)):
    if k == idx_of_largest:
        volume_dictionary[k] = largest_volume
    else:
        volume_dictionary[k] = int(largest_volume * 0.9)
        
volume_dictionary

{0: 2606, 1: 2606, 2: 2896, 3: 2606, 4: 2606}

In [17]:
oversample = SMOTE(sampling_strategy = volume_dictionary)
X, y = oversample.fit_resample(X, y)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.3191037 , -0.41371654, -0.39955547, ..., -0.29360235,
         0.19596913,  0.08113375],
       [-0.24427294, -0.41323502, -0.38847629, ...,  0.25215588,
         0.15447155,  0.49630476],
       [-0.37525554,  0.09799538,  0.46529787, ..., -0.44855562,
        -0.45653482,  0.9030356 ],
       ...,
       [-0.30994718, -0.40808372, -0.39869147, ..., -0.10191157,
        -0.87063403,  0.06624984],
       [-0.4965516 , -0.40089167, -0.41267482, ..., -0.29602238,
         0.63268662, -0.06433178],
       [-0.29688865, -0.41468138, -0.39521766, ..., -0.05294227,
         0.1869491 , -0.02584584]])

In [19]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 26):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, len(unique_labelpairs))
        )
    def forward(self, x):
        return self.network(x)

In [20]:
model = Classifier().to(device)

n_epochs = 20000 #2000
batch_size = 300 #200
lr = 0.0001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr, weight_decay = 1e-5)

train_features = torch.tensor(X_train).to(device)
train_labels = torch.tensor(y_train).to(device)
test_features = torch.tensor(X_test).to(device)
test_labels = torch.tensor(y_test).to(device)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(y_test), shuffle = True)

In [22]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, label = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, label.long()) 
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    #print(f'Epoch {epoch + 1}, Loss: {total_loss:.3f}, Final Batch Loss: {loss.item():.3f}, Penalty: {penalty:.3f}')
    print(f'Epoch {epoch + 1}, Loss: {total_loss:.3f}, Final Batch Loss: {loss.item():.3f}')

Epoch 1, Loss: 59.361, Final Batch Loss: 1.677
Epoch 2, Loss: 59.113, Final Batch Loss: 1.641
Epoch 3, Loss: 58.868, Final Batch Loss: 1.611
Epoch 4, Loss: 58.628, Final Batch Loss: 1.612
Epoch 5, Loss: 58.399, Final Batch Loss: 1.625
Epoch 6, Loss: 58.105, Final Batch Loss: 1.628
Epoch 7, Loss: 57.754, Final Batch Loss: 1.594
Epoch 8, Loss: 57.317, Final Batch Loss: 1.559
Epoch 9, Loss: 56.771, Final Batch Loss: 1.588
Epoch 10, Loss: 56.092, Final Batch Loss: 1.539
Epoch 11, Loss: 55.438, Final Batch Loss: 1.560
Epoch 12, Loss: 54.715, Final Batch Loss: 1.553
Epoch 13, Loss: 53.986, Final Batch Loss: 1.498
Epoch 14, Loss: 53.407, Final Batch Loss: 1.477
Epoch 15, Loss: 52.854, Final Batch Loss: 1.428
Epoch 16, Loss: 52.465, Final Batch Loss: 1.477
Epoch 17, Loss: 52.023, Final Batch Loss: 1.446
Epoch 18, Loss: 51.718, Final Batch Loss: 1.484
Epoch 19, Loss: 51.427, Final Batch Loss: 1.430
Epoch 20, Loss: 51.104, Final Batch Loss: 1.388
Epoch 21, Loss: 50.810, Final Batch Loss: 1.416
E

Epoch 171, Loss: 28.050, Final Batch Loss: 0.772
Epoch 172, Loss: 28.123, Final Batch Loss: 0.845
Epoch 173, Loss: 27.959, Final Batch Loss: 0.824
Epoch 174, Loss: 27.877, Final Batch Loss: 0.727
Epoch 175, Loss: 27.771, Final Batch Loss: 0.752
Epoch 176, Loss: 27.844, Final Batch Loss: 0.837
Epoch 177, Loss: 27.469, Final Batch Loss: 0.712
Epoch 178, Loss: 27.743, Final Batch Loss: 0.809
Epoch 179, Loss: 27.629, Final Batch Loss: 0.881
Epoch 180, Loss: 27.518, Final Batch Loss: 0.822
Epoch 181, Loss: 27.594, Final Batch Loss: 0.823
Epoch 182, Loss: 27.563, Final Batch Loss: 0.678
Epoch 183, Loss: 27.418, Final Batch Loss: 0.710
Epoch 184, Loss: 27.551, Final Batch Loss: 0.804
Epoch 185, Loss: 27.526, Final Batch Loss: 0.768
Epoch 186, Loss: 27.400, Final Batch Loss: 0.794
Epoch 187, Loss: 27.563, Final Batch Loss: 0.798
Epoch 188, Loss: 27.643, Final Batch Loss: 0.674
Epoch 189, Loss: 27.310, Final Batch Loss: 0.749
Epoch 190, Loss: 27.318, Final Batch Loss: 0.775
Epoch 191, Loss: 27.

Epoch 339, Loss: 24.900, Final Batch Loss: 0.719
Epoch 340, Loss: 24.768, Final Batch Loss: 0.578
Epoch 341, Loss: 24.761, Final Batch Loss: 0.828
Epoch 342, Loss: 24.532, Final Batch Loss: 0.671
Epoch 343, Loss: 24.765, Final Batch Loss: 0.647
Epoch 344, Loss: 24.643, Final Batch Loss: 0.721
Epoch 345, Loss: 24.923, Final Batch Loss: 0.750
Epoch 346, Loss: 24.574, Final Batch Loss: 0.637
Epoch 347, Loss: 24.518, Final Batch Loss: 0.773
Epoch 348, Loss: 24.786, Final Batch Loss: 0.653
Epoch 349, Loss: 24.496, Final Batch Loss: 0.532
Epoch 350, Loss: 24.782, Final Batch Loss: 0.754
Epoch 351, Loss: 24.531, Final Batch Loss: 0.593
Epoch 352, Loss: 24.805, Final Batch Loss: 0.677
Epoch 353, Loss: 24.681, Final Batch Loss: 0.596
Epoch 354, Loss: 24.523, Final Batch Loss: 0.684
Epoch 355, Loss: 24.693, Final Batch Loss: 0.731
Epoch 356, Loss: 24.578, Final Batch Loss: 0.721
Epoch 357, Loss: 24.501, Final Batch Loss: 0.670
Epoch 358, Loss: 24.639, Final Batch Loss: 0.725
Epoch 359, Loss: 24.

Epoch 507, Loss: 23.570, Final Batch Loss: 0.715
Epoch 508, Loss: 23.718, Final Batch Loss: 0.691
Epoch 509, Loss: 23.539, Final Batch Loss: 0.665
Epoch 510, Loss: 23.584, Final Batch Loss: 0.607
Epoch 511, Loss: 23.636, Final Batch Loss: 0.697
Epoch 512, Loss: 23.477, Final Batch Loss: 0.631
Epoch 513, Loss: 23.673, Final Batch Loss: 0.622
Epoch 514, Loss: 23.531, Final Batch Loss: 0.684
Epoch 515, Loss: 23.608, Final Batch Loss: 0.639
Epoch 516, Loss: 23.646, Final Batch Loss: 0.557
Epoch 517, Loss: 23.496, Final Batch Loss: 0.691
Epoch 518, Loss: 23.525, Final Batch Loss: 0.560
Epoch 519, Loss: 23.586, Final Batch Loss: 0.582
Epoch 520, Loss: 23.469, Final Batch Loss: 0.666
Epoch 521, Loss: 23.605, Final Batch Loss: 0.629
Epoch 522, Loss: 23.590, Final Batch Loss: 0.680
Epoch 523, Loss: 23.443, Final Batch Loss: 0.602
Epoch 524, Loss: 23.429, Final Batch Loss: 0.695
Epoch 525, Loss: 23.619, Final Batch Loss: 0.658
Epoch 526, Loss: 23.414, Final Batch Loss: 0.669
Epoch 527, Loss: 23.

Epoch 675, Loss: 22.772, Final Batch Loss: 0.589
Epoch 676, Loss: 22.958, Final Batch Loss: 0.773
Epoch 677, Loss: 22.942, Final Batch Loss: 0.686
Epoch 678, Loss: 22.933, Final Batch Loss: 0.633
Epoch 679, Loss: 22.924, Final Batch Loss: 0.574
Epoch 680, Loss: 22.970, Final Batch Loss: 0.583
Epoch 681, Loss: 22.925, Final Batch Loss: 0.563
Epoch 682, Loss: 23.060, Final Batch Loss: 0.745
Epoch 683, Loss: 23.113, Final Batch Loss: 0.714
Epoch 684, Loss: 22.929, Final Batch Loss: 0.575
Epoch 685, Loss: 22.984, Final Batch Loss: 0.574
Epoch 686, Loss: 22.820, Final Batch Loss: 0.622
Epoch 687, Loss: 22.876, Final Batch Loss: 0.632
Epoch 688, Loss: 23.004, Final Batch Loss: 0.634
Epoch 689, Loss: 22.937, Final Batch Loss: 0.539
Epoch 690, Loss: 22.890, Final Batch Loss: 0.744
Epoch 691, Loss: 22.780, Final Batch Loss: 0.604
Epoch 692, Loss: 22.853, Final Batch Loss: 0.622
Epoch 693, Loss: 22.683, Final Batch Loss: 0.567
Epoch 694, Loss: 22.968, Final Batch Loss: 0.519
Epoch 695, Loss: 22.

...

Epoch 19225, Loss: 16.055, Final Batch Loss: 0.495
Epoch 19226, Loss: 15.862, Final Batch Loss: 0.375
Epoch 19227, Loss: 16.285, Final Batch Loss: 0.454
Epoch 19228, Loss: 16.113, Final Batch Loss: 0.389
Epoch 19229, Loss: 16.086, Final Batch Loss: 0.330
Epoch 19230, Loss: 16.028, Final Batch Loss: 0.379
Epoch 19231, Loss: 15.969, Final Batch Loss: 0.525
Epoch 19232, Loss: 15.891, Final Batch Loss: 0.469
Epoch 19233, Loss: 15.869, Final Batch Loss: 0.463
Epoch 19234, Loss: 16.152, Final Batch Loss: 0.442
Epoch 19235, Loss: 15.950, Final Batch Loss: 0.453
Epoch 19236, Loss: 15.843, Final Batch Loss: 0.475
Epoch 19237, Loss: 16.022, Final Batch Loss: 0.490
Epoch 19238, Loss: 16.143, Final Batch Loss: 0.471
Epoch 19239, Loss: 15.925, Final Batch Loss: 0.410
Epoch 19240, Loss: 16.048, Final Batch Loss: 0.436
Epoch 19241, Loss: 15.902, Final Batch Loss: 0.382
Epoch 19242, Loss: 15.922, Final Batch Loss: 0.415
Epoch 19243, Loss: 15.754, Final Batch Loss: 0.325
Epoch 19244, Loss: 16.208, Fina

Epoch 19387, Loss: 15.871, Final Batch Loss: 0.452
Epoch 19388, Loss: 16.211, Final Batch Loss: 0.458
Epoch 19389, Loss: 16.129, Final Batch Loss: 0.448
Epoch 19390, Loss: 15.671, Final Batch Loss: 0.389
Epoch 19391, Loss: 15.922, Final Batch Loss: 0.507
Epoch 19392, Loss: 16.315, Final Batch Loss: 0.527
Epoch 19393, Loss: 15.904, Final Batch Loss: 0.494
Epoch 19394, Loss: 15.703, Final Batch Loss: 0.439
Epoch 19395, Loss: 16.018, Final Batch Loss: 0.504
Epoch 19396, Loss: 15.872, Final Batch Loss: 0.379
Epoch 19397, Loss: 16.207, Final Batch Loss: 0.347
Epoch 19398, Loss: 16.057, Final Batch Loss: 0.547
Epoch 19399, Loss: 15.970, Final Batch Loss: 0.516
Epoch 19400, Loss: 15.895, Final Batch Loss: 0.379
Epoch 19401, Loss: 15.879, Final Batch Loss: 0.417
Epoch 19402, Loss: 15.978, Final Batch Loss: 0.422
Epoch 19403, Loss: 16.077, Final Batch Loss: 0.442
Epoch 19404, Loss: 15.627, Final Batch Loss: 0.332
Epoch 19405, Loss: 16.294, Final Batch Loss: 0.396
Epoch 19406, Loss: 16.137, Fina

Epoch 19549, Loss: 16.187, Final Batch Loss: 0.501
Epoch 19550, Loss: 15.916, Final Batch Loss: 0.473
Epoch 19551, Loss: 16.138, Final Batch Loss: 0.495
Epoch 19552, Loss: 16.009, Final Batch Loss: 0.531
Epoch 19553, Loss: 16.126, Final Batch Loss: 0.378
Epoch 19554, Loss: 15.901, Final Batch Loss: 0.536
Epoch 19555, Loss: 15.914, Final Batch Loss: 0.512
Epoch 19556, Loss: 16.091, Final Batch Loss: 0.577
Epoch 19557, Loss: 15.979, Final Batch Loss: 0.400
Epoch 19558, Loss: 15.930, Final Batch Loss: 0.435
Epoch 19559, Loss: 16.345, Final Batch Loss: 0.414
Epoch 19560, Loss: 16.183, Final Batch Loss: 0.470
Epoch 19561, Loss: 16.063, Final Batch Loss: 0.414
Epoch 19562, Loss: 16.094, Final Batch Loss: 0.430
Epoch 19563, Loss: 16.099, Final Batch Loss: 0.356
Epoch 19564, Loss: 15.952, Final Batch Loss: 0.464
Epoch 19565, Loss: 15.858, Final Batch Loss: 0.366
Epoch 19566, Loss: 16.060, Final Batch Loss: 0.455
Epoch 19567, Loss: 16.227, Final Batch Loss: 0.527
Epoch 19568, Loss: 15.932, Fina

Epoch 19711, Loss: 15.788, Final Batch Loss: 0.426
Epoch 19712, Loss: 16.033, Final Batch Loss: 0.499
Epoch 19713, Loss: 16.036, Final Batch Loss: 0.421
Epoch 19714, Loss: 15.844, Final Batch Loss: 0.469
Epoch 19715, Loss: 16.000, Final Batch Loss: 0.441
Epoch 19716, Loss: 16.011, Final Batch Loss: 0.390
Epoch 19717, Loss: 15.801, Final Batch Loss: 0.470
Epoch 19718, Loss: 16.029, Final Batch Loss: 0.479
Epoch 19719, Loss: 15.606, Final Batch Loss: 0.455
Epoch 19720, Loss: 15.888, Final Batch Loss: 0.436
Epoch 19721, Loss: 15.771, Final Batch Loss: 0.483
Epoch 19722, Loss: 15.941, Final Batch Loss: 0.411
Epoch 19723, Loss: 15.968, Final Batch Loss: 0.454
Epoch 19724, Loss: 16.045, Final Batch Loss: 0.371
Epoch 19725, Loss: 15.986, Final Batch Loss: 0.409
Epoch 19726, Loss: 16.267, Final Batch Loss: 0.420
Epoch 19727, Loss: 16.208, Final Batch Loss: 0.566
Epoch 19728, Loss: 15.642, Final Batch Loss: 0.437
Epoch 19729, Loss: 15.540, Final Batch Loss: 0.380
Epoch 19730, Loss: 15.830, Fina

Epoch 19873, Loss: 16.017, Final Batch Loss: 0.382
Epoch 19874, Loss: 16.204, Final Batch Loss: 0.453
Epoch 19875, Loss: 16.059, Final Batch Loss: 0.376
Epoch 19876, Loss: 15.970, Final Batch Loss: 0.438
Epoch 19877, Loss: 15.881, Final Batch Loss: 0.383
Epoch 19878, Loss: 16.069, Final Batch Loss: 0.451
Epoch 19879, Loss: 15.888, Final Batch Loss: 0.362
Epoch 19880, Loss: 15.851, Final Batch Loss: 0.463
Epoch 19881, Loss: 15.998, Final Batch Loss: 0.490
Epoch 19882, Loss: 16.355, Final Batch Loss: 0.486
Epoch 19883, Loss: 15.938, Final Batch Loss: 0.512
Epoch 19884, Loss: 16.139, Final Batch Loss: 0.456
Epoch 19885, Loss: 15.946, Final Batch Loss: 0.447
Epoch 19886, Loss: 16.133, Final Batch Loss: 0.436
Epoch 19887, Loss: 15.960, Final Batch Loss: 0.386
Epoch 19888, Loss: 16.260, Final Batch Loss: 0.517
Epoch 19889, Loss: 15.784, Final Batch Loss: 0.532
Epoch 19890, Loss: 15.756, Final Batch Loss: 0.475
Epoch 19891, Loss: 16.225, Final Batch Loss: 0.540
Epoch 19892, Loss: 16.058, Fina

In [23]:
softmax = nn.Softmax(dim = 1)

for batch in test_loader:
    with torch.no_grad():   
        features, labels = batch
        _, preds = torch.max(softmax(model(features.float())), dim = 1)
        print(metrics.classification_report(labels.cpu(), preds.cpu(), zero_division = 0))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92       518
           1       0.97      0.98      0.97       512
           2       0.63      0.77      0.69       608
           3       0.73      0.55      0.63       487
           4       0.81      0.77      0.79       539

    accuracy                           0.80      2664
   macro avg       0.81      0.80      0.80      2664
weighted avg       0.81      0.80      0.80      2664

